<a href="https://colab.research.google.com/github/luislauriano/Data_Science/blob/master/Projeto_construcao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dicas do Meigarom (YTB)** 

Luis, vamos lá então.

* O projeto que eu mais gostei foi o de Detecção de Cancer de Mama.
* Para resolve qualquer projeto de ML você precisa de 10 passos e você fez 5 deles. Conferir os passos de um projeto de ML novamente
* Gostei das suas explicações depois de cada código.
* Você poderia ter feito uma análise estatística descritiva dos dados, para dar um visão geral dos dados, em termos de métricas estatísticas ( média, mediana, std, moda, … )
* Você poderia ter feito mais análise exploratória de dados. Quais as features mais importantes para o modelo? Qual o impacto de cada feature na variável resposta?
* Você poderia explicar porque usou StandardScaler nas Features. Porque padronizar os dados? O que isso ajuda no modelo?
* Porque você usou LabelEncoder na variável resposta? O que isso traz de benefício?
* Poderia explicar, porque voce usou Random Forest. Qual o motivo da sua escolha?
* Explicar mais sobre as métricas, o que significa cada métrica? Qual das métricas eu deveria olhar? Todas? Apenas Uma?
13h52
* Vai se aprofundando nos projetos e procurando responder porque você está colocando cada coisa. Porque Random forest? Porque Analise Exploratoria? Porque StandarScaler? Para que serve a matrix de correlação?
* Vai fazendo testes e tentando entender o real motivo de cada linha código que você coloca, e porque essa linha é importante e no que ela ajuda?

# **Projetos como base:**

### Projeto do Rafael
https://github.com/barbosarafael/Projetos/blob/master/iNeuron_Back_order_prediction_Notebook/hackaton_ineuron_back_order_prediction.ipynb

### Projeto do Carlos Melo
https://github.com/carlosfab/data_science/blob/master/Setembro_Amarelo.ipynb

### Projeto do Cancer de Mama 
https://github.com/luislauriano/Data_Science/blob/master/Machine_Learning_para_Detec%C3%A7%C3%A3o_do_C%C3%A2ncer_de_Mama.ipynb

#**Referencias:**

Limpeza e preparação dos dados: https://medium.com/@lucasoliveiras/limpeza-e-prepara%C3%A7%C3%A3o-dos-dados-com-pandas-856e844abfbb

Tutorial API do Spotify: https://dandaramcsousa.github.io/2020/05/20/tutorial-api-spotify.html

Escrever e detalhar o maximo e tudo possivel com calma. (Isso também vai ser importante para aprender)   

Explicar cada coisa usada como quando eu for usar o sklearn (para isso vou precisar pesquisar sobre metric score, acuracia e etc para falar sobre)

___________________________________________________________________________________________________


# **Coletando e visualizando dados do Spotify de NexoAnexo**

## **Objetivo do projeto (Solução para o problema)**

contexto

## **Etapas do projeto**

* Coleta do dados 
   1. Api do spotify
   1. Leitura do Arquivo
   2. Importar os pacotes e dataset necessários
   3. Dicionario dos dados
* Limpeza e preparação dos dados
    1. Deletar colunas
    2. Renomear colunas 
    3. Verificar dados ausentes
    4. Verificar os tipos das colunas
    5. Alterar os tipos de colunas 
    6. Outliers
* Exploração e visualização dos dados 
* Etapas do Machine Learning workflow

## **Coleta dos dados**



contexto

### **Explicar toda etapa da coleta dos dados na API do Spotify**

1. Coletar as informações de um artista, dado um nome;
2. Coletar os ids e nomes de todos os álbuns de um artista;
3. Coletar informações de todas as músicas de um álbum;
4. Coletar informações de todas as músicas de todos os álbuns de um artista;
5. Converter tudo para um csv.


In [21]:
!pip install spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import sys
import csv
 
sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials(client_id="dcffc14978ed497aa58834a7864a99b7", client_secret="e8c5cc78c8e74ccd92323d146c635b47"))

In [0]:
def get_artist(name):
    results = sp.search(name)
    items = results['tracks']['items']
    if len(items) > 0:
        return items[0]['artists'][0]
    else:
        return None

In [0]:
def get_artist_id(artist):
    return artist['id']

In [0]:
def get_artist_albums_id_names(id):
  albums = sp.artist_albums(id, country = 'BR', limit=3)
  albums_id_name = {}
  for i in range(len(albums['items'])):
    id = albums['items'][i]['id']
    name = albums['items'][i]['name']
    albums_id_name[id] = name
 
  return albums_id_name

In [0]:
def get_album_songs(album_id, album_name):
  spotify_album = {}
 
  tracks = sp.album_tracks(album_id)
  
  for n in range(len(tracks['items'])):
    id_track = tracks['items'][n]['id']
    track = sp.track(id_track)
    spotify_album[id_track] = {}
    
    spotify_album[id_track]['album'] = album_name
    spotify_album[id_track]['album_type'] = track['album']['album_type']
    spotify_album[id_track]['track_number'] = track['track_number']
    spotify_album[id_track]['id_track'] = track['id']
    spotify_album[id_track]['name'] = track['name']
    spotify_album[id_track]['popularity'] = track['popularity']
    spotify_album[id_track]['explicit'] = track['explicit']
    spotify_album[id_track]['duration_ms'] = track['duration_ms']
    spotify_album[id_track]['release_date'] = track['album']['release_date']
 
    artists_track = track['artists']
    spotify_album[id_track]['artists'] = []
    for artist in artists_track:
      spotify_album[id_track]['artists'].append(artist['name'])
  return spotify_album


In [0]:
def get_all_albums_songs(albums_ids_names):
  spotify_albums = []
  albums_names = []
  for id, name in albums_ids_names.items():
    if name not in albums_names:
      albums_names.append(name)
      album_songs = get_album_songs(id,name) 
    for item in album_songs.items():
      spotify_albums.append(item[1]) 
  return spotify_albums

In [0]:
def convert_to_csv(filepath, name):
  keys = filepath[0].keys()
  print(keys)
  csv_name = ''+ name + '.csv'
  with open(csv_name, 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(filepath)
  return

In [29]:
name = "NexoAnexo"
artist = get_artist(name)    
if artist:
  artist_id = get_artist_id(artist)
  albums_id_names = get_artist_albums_id_names(artist_id)
  all_albums = get_all_albums_songs(albums_id_names)
  convert_to_csv(all_albums, 'NexoAnexo')   
else:
  logger.error("Can't find artist: %s", artist)

dict_keys(['album', 'album_type', 'track_number', 'id_track', 'name', 'popularity', 'explicit', 'duration_ms', 'release_date', 'artists'])


### **Dicionario dos dados**

contexto 

**Informações sobre os atributos**


### **Importar os pacotes e datasets necessário**



contexto

In [32]:
#importar os pacotes necessarios
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# configurar o notebook
sns.set_style()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### **Leitura do arquivo**

Primeiro vamos ler nosso arquivo fora do notebook e tirar algumas informações, de como os dados estão separados e etc..

In [0]:
dados = 'NexoAnexo.csv'

df = pd.read_csv(dados, sep = ',')

contexto

In [0]:
print(f'Número de linhas ou variaveis {df.shape[0]}')
print(f'Número de entradas ou colunas {df.shape[1]}')

Número de linhas ou variaveis 9879
Número de entradas ou colunas 40


contexto

## **Limpeza e preparação dos dados**

contexto

### **Deletando colunas**

Como o nosso conjunto de dados contém colunas que não serão uteis

In [0]:
df.head()

,gameId,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,blueTowersDestroyed,blueTotalGold,blueAvgLevel,blueTotalExperience,blueTotalMinionsKilled,blueTotalJungleMinionsKilled,blueGoldDiff,blueExperienceDiff,blueCSPerMin,blueGoldPerMin,redWardsPlaced,redWardsDestroyed,redFirstBlood,redKills,redDeaths,redAssists,redEliteMonsters,redDragons,redHeralds,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,4519157822,0,28,2,1,9,6,11,0,0,0,0,17210,6.6,17039,195,36,643,-8,19.5,1721.0,15,6,0,6,9,8,0,0,0,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,4523371949,0,12,1,0,5,5,5,0,0,0,0,14712,6.6,16265,174,43,-2908,-1173,17.4,1471.2,12,1,1,5,5,2,2,1,1,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,4521474530,0,15,0,0,7,11,4,1,1,0,0,16113,6.4,16221,186,46,-1172,-1033,18.6,1611.3,15,3,1,11,7,14,0,0,0,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,4524384067,0,43,1,0,4,5,5,1,0,1,0,15157,7.0,17954,201,55,-1321,-7,20.1,1515.7,15,2,1,5,4,10,0,0,0,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,4436033771,0,75,4,0,6,6,6,0,0,0,0,16400,7.0,18543,210,57,-1004,230,21.0,1640.0,17,2,1,6,6,7,1,1,0,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4


## **Exploração de Visualização dos dados**

Colocar todas as etapas do projeto de lol 

Depois de organizar o que gostaríamos de fazer e mexer na API do Spotify para ter um .csv bonito, nos falta apenas as visualizações dos dados. Porém, entretanto, todavia, ainda não disse quais perguntas vamos responder. Então vamos lá:

Qual é a música mais popular de Nexo?
Qual é o álbum mais popular?
E o single?

Apostas: 

No album Real Plug, 
No album Trap de Cria, acredito que seja OI Fake 
No album TFF, acredito que seja Match, por mais que a minha preferida seja ...

Musica mais pouplar, Ratatah 
Album mais popular, TFF


Ver se a popularidade está funcionando da maneira correta

Lembrem da definição de popularidade que foi dada lá no volume 01. Uma música nova tem muito mais chance de ser mais popular. 

O sucesso Trap de cria

In [0]:
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials(client_id="dcffc14978ed497aa58834a7864a99b7", client_secret="e8c5cc78c8e74ccd92323d146c635b47"))

if len(sys.argv) > 1:
    name = ' '.join(sys.argv[1:])
else:
    name = 'NexoAnexo'

results = sp.search(q='artist:' + name, type='artist')
items = results['artists']['items']
if len(items) > 0:
    artist = items[0]
    print(artist['name'], artist['images'][0]['data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIAJcAlwMBIgACEQEDEQH/xAAcAAAABwEBAAAAAAAAAAAAAAAAAgMEBQYHAQj/xAA6EAACAQMCBAQDBgQFBQAAAAABAgMABBEFIQYSMUETIlFxB2GBFCMyUpGhQkOx0RUzweHwCCRygpL/xAAaAQACAwEBAAAAAAAAAAAAAAAAAgEDBAUG/8QAIBEAAwACAwADAQEAAAAAAAAAAAECAxESITEEIkETUf/aAAwDAQACEQMRAD8ArPE/Anh2fiacN4+kfc/Ws8nheCUxyoUddip7Vtmqa2zSNb2+nvzA7PLMiKf3rLuL8ya3I/hGIlVyjEHBx6is+Cq8o254j1Fy+FXFdvY28ulardJDGG5rZ5DgZJOVz77/AFrWYXWZQ8TB1PRlOQfrXmB0+6wKeaLxFq+gvnTbySJc5KbMh91O1WVi29opWTXTPTA2pQHasq0H4uxOEi12zaP1mt9x9V7fStD0jW9M1mES6beRzrjdVPmX3HWqHDQ6pMlFbNHGwpNTR+1QiWHFdri0bNMQADNDG9dFCggFChQoAKRRcUoRRSKBhM0Rs4pRhRDSkiRBrtdIzQqAM2vOFLPWoVkmll5mPMGQjbpt+1UviTR49PmeFWZlj5TljknNadYXsXgkQyB8dvSs14zu2l1a4X+A4xj2FR8fI3WmXZ4SWyt3ACB5BsCoxTKIhyeYdaeai3/bn9PoKjI23xW9HPodvahvwdB39aLH9ptJllhd45F3V0JBB9xSkEhGysRmpGCeNkYOoPlABPb/AJmmaTBE3o/xP4h07ljunjvYR2nXDf8A0P8AWrVB8Y7ZprcS6TMkZyJuWUNj0K7D96oBtLF25HdUfOD8vnUW9qrvL4J8iHCn1pKxyMqo9OaTqMOp6fBe2xHgzqHTffHz+fyp/mvMeh8Qa3oM3NptzJGuctEfMjfQ7fpWmcP/ABWWXlh1yxaJsb3EG6n/ANev6Vnc6LU9mo9qHamGmarYarD4unXcVwmN+Rtx7jqKfdqgDtCuZ3xQz+lQB0VyhnahQAU0Q0dvSidM1DRKCMKFdNClGM4vtZsbNWAkR3H8Kbn9qznV7tbq+e4YbMcgVbtesobW3MUCrHGdzyAAuegFUSfKMDghaX42vUWZ2/BLVMCLwh0XA96iUFP7glgD+tMjgGt8mKhVSaUZyBikEejrknc1LYStiodzuTSqgE4zsNyKb42z0owyI++e/tVTRcLCbOQrHA3AJpe3l5iAc/Q9KZInITjB8valYAx3UkHptS0kMn2P0lurK5Fxp9w8Ew/ijYrvV04c+LN9alYNdg+1RA48WMcsn1HQ1RVLcgHUjqfWjpZrfMqx58ZzhR9OtEd9MW1rtG/6Pxpw9rBAtNRjWVv5Ux5G/erB6Y39q8qXFlcWrnnjZd9tqm9A451/QmVbe7aSAHeGbLr9O4+lM8X+Cq0ejyRXM+lZvoXxa0y75Y9WtpbNz1lXzJ/cVfrG/s9RgWexuYriFhnnicMP9qqaa9H2h1miZoZopNQSA9aFFJoUoGXa0Zpbc28kJPI+VZd89f8AaqVqloYoliKEkHI9T9K0y4hPIdhVQ4ghSGCRio2z19ayYL09GrLG1soc6tzkYwO4prJGwfDggjrUtdxGO48QDKMN8GpPVbP7dZLcBQJFXOQOtdL+iWtmHhtPRVAMNgHmHY0tGfWkyVB2XFKxebGKsoWRSJWJIXqelKtGy4BH4RTrT7cmVDv64HWnGo5ln8R15euwXBqh5O9GhR1sjQoGMjfHWjRgrHzKcdMUflyOVffI7UTcLknsAAKnZGgykFQ4PQ4NHW6NsedSQ3Ntikhuh+ZpK5GZEIJwd/lTQvsRb6LKNYjuoVjuokbG3N3pld2tpcDMZCnBqHEpAAz0oGZsda0sy7CzWxR2welK6fqWo6XMJtPupoHB6xsRn3Hem7XB33pexeJ5vvvw42pWkw2XzQ/i3qVvyx6vbxXadPEjHI/9jWhaHxvoOt8qW94sMx/k3GEb6etYHc2qHLwny52pkVZDscEVXWJMsVtHqnnBGx/ShXnXReNdf0ccltfNJFjHhT/eKPbNcql4WOsiNdQeJCWxkVWOJhH9jaNlzzsAauL27afbfZ5Tl0yrGqjf6fe6ndBQjJCudz/FXKjq9HRruSr3kMSWgDbnAx70jd3iwaZ5Dty+XNLa0PBvZLQOrBVG47H/AIakNA4G1Hifl5ma006P+cyZaT/xH+prpzHLWzFVcdmdt5lyetGjnZB0H1re9N+EfDtsgN0lzdSjvLJgfouKfSfDThVhn/C0HtIwrXoyLZnfw10E6oJpLg8sDRfiXqDnbHpR9b4QvbDU7eONJLqCQjDZ3G/T/erbp/g8PXd3okWnvAsMnMsiy845G/Bud+nr0qehZ5grynm2yDXMyW5tnUxyqhGY3nDSadqiW8zAhonZiAfI2NhVes9PkubsQLgsx5QeX1rWtUtYGl+1tGhuANn79MVUlt5LV7u8tYmR1VmWVSBhvrRjum9BaSRQtUVrecxMpUqxDKdjTB3AOxOOwPanGq38+oXb3F04eZgA7KMAt3NR7HJFdGJ0jnXe30LB6Dk8tJHIbHX50oqg9SacQTUAt5m5R69aNHnmAFLMkXMMbDYe9Gbwoiwj39DUkB2nKpyA7fIUgZM9aTY0UmoJDkihSRahRsNnpi8lgtdJs9Y1+7js4JLWOSQyHzF+QZAXqTn0rMNe4k1ji25a24ZtprXS4WOGHkaQ/md+3tVd474oueLNflvpS626eS2hJ2jT+56mleA7i/8A8U+wW15LBbTAmREAPOQNgM9DVLxRj3f6WrJValvosPBOhGbiKCK/jItoF8WVpf5pA+fbNbhYPDOgWFV8Nfy9KoR8C2TllzHKrYDPhl/YbU5itHsbi2mtJ3Lgq8vheVY8nJHbNVLP1vRa8KfjL/yD8uK6UCjI3pVcOoYdCAR7UCOg9TWsyFC+ImnJHe6ffpIkUkh8FsbFiNxv8t6bw2z26CQTPjHmDOCpqzcY6NBq2nxpdr/luGjZWKsjdM5FVO24Tisp5mkurm6iyfDSZ8hfTYdfrWD5MLls3/HyfVITmc3h8OA7HYykbY+XrUPr9uh02W1jGECEsxGas3htGnIikHoKY22myXss5YeQZ5nPas8b2X2+jD9T02fTNQls7jHNH0bsynow+RFESKIqFzg9zW58R/Ddda4et28Tk1SIN4crHYgnIVvlv9M1heoWt1pl5LZ3sTRXETFXRuxrrS+jlPphZY0VyM57UmT6UmXOdzmuc1TsgPzURm32ro36ZPsKIBk+9AB9yOtd5WC8x3GetXThbgeXU0Se8Zo42GQvTau8b8KW2iRWslnzBXJWTLZz3zVP9Z3ovWCuPIpAAJ2612nk1iyYIXqO1Cn2V8WFtYHuZljTYdz6CtY+E9jp8OuCOdUOYW2cdTtU7wj8LdK05Yp9RV7q75fPzORHn5L/AHq08T6QF0VYdKtoEnSRfATkGAScdMYPXvSZ8dZFrY2O1PojxHb2EQ5YLiaCeIc6ssXjBd++QcfU0x0awbWb95bm4vJ4DvLIfuwx6ADAH7VOafwpY2dnFDK080nKBITIQJD8wO1S9uAimMACNfwAfwj0qrF8bj6WPMtaQ4AwoCjAFA/0ow6VwVrMwnJyyIQ/fY5G1NptNRx903KfQ9Kc3EbPBIgJyykDHrRbWXnt0ZtmwOYehpKlP0aac+EU+kPkklAMdetOrHTI4IFEgVm5uZsDGfTPrT45Yg5xjtR+29LOKF+DVlql2JSDJOBsOmazv4o8Dx8RWbX1hFy6nCMpj+YPyn/T0rR/akJgfXr3q0rPHBicOUKkODgqeoPpRpI2ByV5c9qu/wAWtJGj8WyzQ4WC9HjKBthujD9d/rVJlnZzkneoAL4jJsCwPvR7Xl+0w83TnGf1pBmLHJoAkHI6ioJRuOiavZCJYhKPEVfNgEhfftUbxhcB7qCWEG4JkCgkZWNd+gqu8HyahqMUcUTKLdVMbr+ZvU1e0SJ1twYt4+VZg3Y+1c61wo6cPnJSzpLzxtDdKnM7GSN1GMeq0Kuuu2i/YFa3A50IIP7f0NCkeSif5yaNPcQaaillLs2cBRvt1/rUba8T2N1qsNs9xbBWz4S+L5+cA5DL2Iwf0proevabxM6W3ieFfxr4kZVs83qR8umR86Sm4Nt+fFvII3EgkEp3YYHt03O3qTXWOZougYZ232ojADBAwOpoluPCjSMtzBQBk9TSpXJPcUChl22NdwO9F5SB2NdBzQB360XlX0FG7UMZ6dfSgAYoYqLvtctoGaKGVHkX8Rzsv19ako2540b1UGgAYFJS4xgdaVx3JpNiB06nvQBhX/UDEE1LSCD5mikyPTdaykRtjNaP8Xr9NY4zNrbETJZxiEY6Fju/7nH0qrtZW/JGkRLSgZkYN5R8sUrY6nZANGykA96Jg1MTWErMzEHcDG1WXQvh9cXkAlu5DGrDYKKrvJMLdDTiqukRPA2vpomoFbgZt5sBj+U9jV/neW9vkeKUfZ3HPs2Kgb34bLEh8C6ct25lzTfTbTVdDJSVhJCGwD6VkyuMn2T7NeHnH1a6L3chbPSx4UfUjYndvnQqEt477XWy7/dKMKEOM470Kzl7eynaC19pnE9pNZSck1u5fxGDFccp643wenuRWjaZ8Q9b5me70uxHM0OAruv4wSf4T+DGG9CRVc0a/g0zVJgs6xzSRRqpZWP4ZY3ZfKM/hRv1q1jWtNW5cveSJ97eMsQt5f5vh8hPl2/C5+XNXXTOS+hUfE/UVWItokP3kcb48Z/IWblKt5NivU/KrDwRxZdcSXV3HdaaLPwY1cEOXyS7qQcqMfgz7EVVbnWdPvbW5EF0VimivkjdIpm/zWyhJ5Md/pgdanNB4y0Wz1BbOa5fxb8x/eSI6qhWGNdywGAzBt/r3qdkF+Xahiubjc96N7VIHP3oZ7EVzJ9j8qTcgnlbegCJ1DQNMuIpF5fCLA4MYHlJ9B2qXgBESJkgKAN+tcUKSQAB7UrQTsK+FBz2qPvLhxBJ4CHm5Tvnfp2p3dsPDGTgE0ykcFiuAy9NqgEefuJeFbrRbQ38pmM7yHxwxzhWGQebv1wfnUWkMcK26hsmY4G/fbNejryzguoWS4gV0YEFGGQQax3iThGwW5mutDmkCWreaFhkYHXl+VVXSn0vhN+DC7hisraOeXBeUhYE747savPDWpKIo4nIdG9Oo96povk1IGZfBkgiQBYWRTyHG2R1Az3BqZ4beNS86KGyuEB6qe+fasmalXZpxLReJjA47DvUDdwwsziQAo3UEbGj20skjEqWf1IpC9hmycqcViqn6aEvwRt7ebTgfsLRSwMdo32K/WhTXxJYmOOh7GhVfIbSMu1K9mF14qMVlyTzZ6bY2pWPUpzbPNNcTySnPWQ9T3J60KFegOPrbELK+vwUhW8mWID8AbAIHb9qdySPI3iM7u7AKPEYtgdutChQPKRsfwp4smuoRompO0rxDltpjlmIHZievvWknbpQoVKK69Clj3G/rSbMeYChQqRRVQMUYHGe9doUAISReKACSAtcS3jjGdyfU0KFKSiD4n1IWkTW8AbxWTLN+Vfl86ol1aTy2NxNbSujRHlZVbBZcg9fb+poUK52dt0dDAtQZvdrKXBjRIFhPIZIwAxGepx1qe0/UGjEVvEX+8wvM5ycd6FCprwhekrd/EbTdEzZWtjLdzREB3JCJnv8/wBqRh+K1jclkvtJliBfCtC4OF9Tnv7UKFalhhz2jO8t8vSwQvY6vZrd6e5eJjgFlKkfQ0KFCuNllTbSOpHc7P/Z'])